# Real Estate: Lead Conversion

## Problem Statement
Develop a scoring system for lead conversion and derive online behavior analysis along with recommendations to client.

## Data
There are two data files - one containing the information on online beavior of the leads and the other containing 
information of those leads being converted to tenants. These two files, henceforth will be addressed as `leads_data` and 
`taregt_data` and the target column `converted_to_tenant` as `output`.

## Approach
The primary tasks to accomplish the goal of this project are:
1. Understand the data
2. Map `target_data` to `leads_data`
3. Develop a scoring mechanism

The solution follows a standard Data Science solution approach targeting the following sections:
* [Exploratory Data Analysis](#eda)
* [Data Cleaning](#data-cleaning)
* [Feature Engineering](#feature-engineering)
* [Model Selection and Training](#model-selection-and-training)
* [Benchmarking Model Results](#benchmarking-model-results)
* [Testing and Scoring](#testing-and-scoring)
* [Conclusion](#conclusion)

# Imports

In [1]:
import os
import random
import numpy as np
import pandas as pd

from functools import partial

# from pandas_profiling import ProfileReport
from pandas.api.types import is_numeric_dtype

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE
random.seed(0)

<a id = "#eda"></a>
# EDA

## Target Data

In [2]:
path_target_raw = "./data/raw/target_data.csv"
target_raw = pd.read_csv(path_target_raw)

In [3]:
# folder_report = "./reports/"
# if not os.path.isdir(folder_report):
#     os.makedirs(folder_report)

# path_report_target = os.path.join(folder_report, "target.html")
# profile = ProfileReport(target_raw)
# profile.to_file(path_report_target)

In [4]:
print("target_data shape: ", target_raw.shape)
target_columns = target_raw.columns.tolist()
for column in target_columns:
    print(f"number of unique values in `{column}`:", target_raw[column].nunique())

target_data shape:  (266046, 2)
number of unique values in `lead_id`: 198858
number of unique values in `converted_to_tenant`: 2


In [5]:
column_id, column_output = target_columns

In [6]:
print(f"unique values in `{column_output}`:", target_raw[column_output].unique())

print(f"value count in `{column_output}`:")
print(target_raw[column_output].value_counts())

unique values in `converted_to_tenant`: [0 1]
value count in `converted_to_tenant`:
0    185723
1     80323
Name: converted_to_tenant, dtype: int64


In [7]:
print(f"duplicate values in column `{column_id}`:", target_raw[column_id].duplicated().sum())
print(f"duplicate rows in target_data:", target_raw.duplicated().sum())

duplicate values in column `lead_id`: 67188
duplicate rows in target_data: 51979


In [8]:
# Checking for lead ID values having more than 1 output
target_groups = target_raw.groupby(column_id)
target_distinct_value = target_groups[column_output].nunique()
print("number of lead ID values having more than 1 output:", len(target_distinct_value[(target_distinct_value > 1)]))

number of lead ID values having more than 1 output: 15209


### Obserations
<ol>
    <li> Some lead ID values have multiple outputs </li>
    <li> There are multiple data points repeating information </li>
</ol>

### To-Do
<ol>
    <li> Drop lead ID values having more than one outputs </li>
    <li> Drop duplicate rows </li>
</ol>

## Leads Data

In [9]:
path_leads_raw = "./data/raw/leads_data.csv"
leads_raw = pd.read_csv(path_leads_raw)

C:\Users\Archisha\AppData\Local\Temp\ipykernel_10984\3576564257.py:2: DtypeWarning: Columns (31,32,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  leads_raw = pd.read_csv(path_leads_raw)


In [10]:
# folder_report = "./reports/"
# if not os.path.isdir(folder_report):
#     os.makedirs(folder_report)

# path_report_leads = os.path.join(folder_report, "leads.html")
# profile = ProfileReport(leads_raw)
# profile.to_file(path_report_leads)

In [11]:
# identifying a column in `leads_data` that resembles `lead_id` column in `target_data`
column_id is leads_raw.columns, leads_raw.filter(like=column_id).columns, leads_raw.filter(like="id").columns

(False,
 Index(['ga_lead_id'], dtype='object'),
 Index(['client_id', 'ga_lead_id'], dtype='object'))

In [12]:
# identifying which column of `ga_lead_id` and `client_id` matches `lead_id` `target_data`
id_target_set = set(target_raw[column_id].unique())
print("unique IDs in `target_data`: ", len(id_target_set))

possible__id_columns = ["ga_lead_id", "client_id"]
for column in possible__id_columns:
    id_leads_set = set(leads_raw[column].unique())
    print(f"unique IDs in `{column}`: ", len(id_leads_set))
    id_common = id_leads_set.intersection(id_target_set)
    print(f"common lead IDs in `target_data` and `{column}`: ", len(id_common))

unique IDs in `target_data`:  198858
unique IDs in `ga_lead_id`:  3843
common lead IDs in `target_data` and `ga_lead_id`:  3497
unique IDs in `client_id`:  2938
common lead IDs in `target_data` and `client_id`:  0


### Column Analysis

#### `ga_lead_id` 

In [13]:
column_id_leads = "ga_lead_id"
column_id_target = column_id

In [14]:
# checking number of times the `column_id_leads` is repeated
leads_raw[column_id_leads].value_counts().value_counts()

1    3553
2     250
3      26
4      10
6       2
5       1
Name: ga_lead_id, dtype: int64

#### `client_id`

In [15]:
column_client = "client_id"

In [16]:
print(f"unique values in column `{column_client}`", leads_raw[column_client].nunique())
print(f"null value percentage in column `{column_client}`", leads_raw[column_client].isna().mean())
print(f"description of number of data points for every `{column_client}`")
leads_raw[column_client].value_counts().describe()

unique values in column `client_id` 2938
null value percentage in column `client_id` 0.0
description of number of data points for every `client_id`


count    2938.000000
mean       99.115044
std       193.699108
min         3.000000
25%        30.000000
50%        58.000000
75%       112.000000
max      6165.000000
Name: client_id, dtype: float64

##### Hypothesis
<ol>
    <li> A client has multiple interactions with the website </li>
    <li> A client might be linked with multiple leads </li>
    <li> Each `lead_id` is associated with only one client </li>
</ol>

In [17]:
leads_client_groups = leads_raw.groupby(column_client)

In [18]:
# hypothesis 1 testing
random_client = random.choice(leads_raw[column_client])
leads_client_sample = leads_client_groups.get_group(random_client)
print(f"number of total interactions for a sample client - {int(random_client)}:", leads_client_sample.shape[0])
print(f"number of unique interactions for a sample client - {int(random_client)}:", leads_client_sample.drop_duplicates().shape[0])
print(f"percentage of duplicate rows for a sample client - {int(random_client)}:", 100 * leads_client_sample.duplicated().mean())

number of total interactions for a sample client - 4855140000000000000: 49
number of unique interactions for a sample client - 4855140000000000000: 34
percentage of duplicate rows for a sample client - 4855140000000000000: 30.612244897959183


In [19]:
# hypothesis 2 testing
client_leads_count = leads_client_groups[column_id_leads].nunique()
print("percentage of clients having more than one leads:", (client_leads_count > 1).mean())

percentage of clients having more than one leads: 0.24642614023144996


In [20]:
# hypothesis 3 testing
leads_lead_groups = leads_raw.groupby(column_id_leads)
leads_client_count = leads_lead_groups[column_client].nunique()
has_more_than_1_client = (leads_client_count > 1)
print("number of leads with more than 1 client:", has_more_than_1_client.sum())
print("percentage of leads with more than 1 client:", 100 * has_more_than_1_client.mean())

number of leads with more than 1 client: 3
percentage of leads with more than 1 client: 0.07808433107756377


##### Hypothesis Results
<ol>
    <li> A client has multiple interactions with the website - positive </li>
    <ul>
        <li> There exists duplicate data points for a client but we do not have enough evidence to drop them </li>
    </ul>
    <li> A client might be linked with multiple leads - positive </li>
    <li> Each `lead_id` is associated with only one client - negative </li>
    <ul>
        <li> We can drop the lead IDs having more than one clients as it constitutes 0.07% of total lead IDs </li>
    </ul>
</ol>

<b> Note </b>
<ul>
    <li> A client ID is associated with more than one lead IDs, we cannot merge data on client ID to generate lead ID features </li>
</ul>

#### Other Columns
Analysing remainning columns of `leads_data` and grouping them based on their characteristics for feature engineering

In [21]:
# function for column analysis
def basic_info(column):
    print(f"--------------{column}--------------")
    print(f"datatype: {leads_raw[column].dtype}")
    print(f"NaN value %: {leads_raw[column].isna().mean()}")
    nunique = leads_raw[column].nunique()
    if nunique <= 5:
        print(f"unique values: {leads_raw[column].unique()}")
        print(f"value counts")
        print(leads_raw[column].value_counts())
    else:
        print(f"number of unique values: {nunique}")
        if is_numeric_dtype(leads_raw[column]):
            print(f"minimum value: {leads_raw[column].min()}")
            print(f"maximum value: {leads_raw[column].max()}")
    print(f"sample client data: {leads_client_groups.get_group(random.choice(leads_raw[column_client]))[column].tolist()[:10]}")
    print(f"--------------------------------------")


In [22]:
other_columns = [column for column in leads_raw.columns if column not in [column_id_leads, column_client]]
for column in other_columns:
    basic_info(column)

--------------sessions--------------
datatype: int64
NaN value %: 0.0
unique values: [0 1 2 3 4]
value counts
0    273049
1     18060
2        78
3         9
4         4
Name: sessions, dtype: int64
sample client data: [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
--------------------------------------
--------------timestamp--------------
datatype: object
NaN value %: 0.7170089285714286
number of unique values: 72
sample client data: ['15/09/2021 8:41', '15/09/2021 8:41', nan, '15/09/2021 8:41', '15/09/2021 8:41', nan, nan, nan, '15/09/2021 8:41', nan]
--------------------------------------
--------------hits--------------
datatype: float64
NaN value %: 0.7170089285714286
number of unique values: 55
minimum value: 1.0
maximum value: 358.0
sample client data: [7.0, nan, nan, nan, nan, nan, 15.0, nan, 4.0, 4.0]
--------------------------------------
--------------unique_events--------------
datatype: int64
NaN value %: 0.0
number of unique values: 24
minimum value: 0
maximum value: 74
sample client da

In [23]:
# grouping columns based on above observation
leads_features_info = {
    "1" : {"columns": ["sessions", "hits", "unique_events",
                       "session_duration", "page_per_session", "enquiries",
                       "pageviews", "time_on_page", "avg_time_on_page",
                       "property_search", "lead_dropped", "bounce_rate",
                       "total_events"
                       ],},
    "2" : {"columns": ["timestamp", ],},
    "3" : {"columns": ["social_source_referral" ],},
    "4" : {"columns": ["contact_us_click", "app_downloads"],},
    "5" : {"columns": ["source", ], "source_mapping": {'google': "google", 'Google': "search", '(direct)': "direct", 'eservices.client.ae': "client", 'client.ae': "client", 'Property Finder': "leasing_website", 'instagram.com': "chat_media", 'PropertyFinder': "leasing_website", 'facebook': "chat_media", 'client.com': "client", 'ecservices.client.ae': "client", 'SMS': "chat_media", 'PF_Desktop': "leasing_website", 'l.facebook.com': "chat_media", 'Bayut': "leasing_website", 'Facebook': "chat_media", 'tpc.googlesyndication.com': "google", 'm.facebook.com': "chat_media", 'bing': "search", 'burpsuite': "leasing_website", 'Email': "email", 'Ffvtabghef': "email", 'clientretail.com': "client", 'Esignature': "email", 'acm2.eim.ae': "email", 'eservicesqa.client.ae': "client", 'teams.microsoft.com': "chat_media", 'c3.avaamo.com': "email", 'Outsource': "leasing_website", 'testsecureacceptance.cybersource.com': "leasing_website", 'mail.google.com': "email", 'facebook.com': "chat_media", 'email signature': "email", 'dreclands.ae': "leasing_website", 'gulfnews': "search", 'clientdistrict.client.ae': "client", 'yahoo': "search", 'tagassistant.google.com': "google", 'm.nearbyme.io': "leasing_website", 'client51.client.ae': "client", 'developers.google.com': "google", 'duckduckgo': "search", 'clientqa.client.com': "client", 'login.microsoftonline.com': "email", 'search-dra.dt.dbankcloud.com': "email", 'analytics.google.com': "google", 'qwant.com': "search", 'Email-clientdistrict-buildingD': "email", 'korter.ae': "leasing_website",'linkedin.com': "chat_media", 'Bayut_Desktop': "leasing_website", 'lm.facebook.com': "chat_media", 'l.instagram.com': "chat_media",}},
    "6" : {"columns": ["device_category"], "devices": [device for device in leads_raw["device_category"].unique().tolist() if device is not np.NaN]},
    "7" : {"columns": ["event_category"], "categories": [cat for cat in leads_raw["event_category"].unique().tolist() if cat is not np.NaN]},
}
# columns to drop which are not providing any relevent information
leads_columns_to_drop = ["contact_us_form", "pdf_download", "website_search", 
                   "social_media_click", "date_hour_minute", "medium",
                   "data_source", "property_view_location",
                   "property_view_price", "property_view_size",
                   "property_view_bedrooms"]
# columns need more analysis than provided by `basic_info` 
leads_columns_need_more_analysis = ["campaign", "event_label", "event_action"]


### Obserations
<ol>
    <li> `lead_id` of `target_data` does not have the same column name in `leads_data` </li>
    <li> There are no common IDs in `target_data` and `client_id`, hence we will use `ga_lead_id` to merge with the `target_data` </li>
    <li> Number of leads are more than number of clients, hence a client might be associated with more than one lead ID </li>
    <li> We have grouped columns based on their characteristics and identified which to drop and those which need advanced analysis </li>
</ol>

### To-Do
<ol>
    <li> Drop lead IDs from `target_data` having more than one client ID </li>
    <li> Features to be created </li>
    <ul>
        <li> Number of leads for a client </li>
        <li> Number of interactions for a client </li>
        <li> Normalize required numerical features by number of leads for a client </li>
        <li> Generate features based on column grouping </li>
    </ul>
    <li> Drop columns identified in column analysis </li>
</ol>

# Data cleaning

## Target Data

In [24]:
# dropping lead ID values having more than one outputs
# target_groups_filtered = target_groups.filter(lambda x: x[col_label].nunique()==1)
lead_ids_to_drop = target_distinct_value[~(target_distinct_value==1)].index.tolist()
target_filtered = target_raw[~target_raw[column_id_target].isin(lead_ids_to_drop)]
print("shape of the `target_data` after dropping lead ID values having more than one outputs:", target_filtered.shape)

shape of the `target_data` after dropping lead ID values having more than one outputs: (226847, 2)


In [25]:
# dropping duplicate rows
target_filtered.drop_duplicates(inplace=True)
print("shape of the `target_data` after dropping duplicate rows:", target_filtered.shape)

shape of the `target_data` after dropping duplicate rows: (183649, 2)


C:\Users\Archisha\AppData\Local\Temp\ipykernel_10984\313814757.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_filtered.drop_duplicates(inplace=True)


In [26]:
# dropping lead ID values having more than one clients in Leads Data
leads_groups = leads_raw.groupby(column_id_leads)
leads_clients = leads_groups[column_client].nunique()
leads_to_drop = leads_clients[(leads_clients>1)].index.tolist()
target_filtered = target_filtered[~(target_filtered[column_id_target].isin(leads_to_drop))]
print("shape of the `target_data` after dropping lead ID values having more than one clients in `leads_data`:", target_filtered.shape)

shape of the `target_data` after dropping lead ID values having more than one clients in `leads_data`: (183646, 2)


In [27]:
# saving filtered Target Data
path_target_filtered = "./data/target.csv"
target_filtered.to_csv(path_target_filtered, index=False)

In [28]:
del target_raw

## Leads Data

In [29]:
# dropping columns 
leads_filtered = leads_raw.drop(columns=leads_columns_to_drop)

In [30]:
leads_filtered["timestamp"] = pd.to_datetime(leads_filtered["timestamp"])

In [31]:
path_leads_filtered = "./data/leads.csv"
leads_filtered.to_csv(path_leads_filtered, index=False)

In [32]:
del leads_raw

# Feature Engineering

## Designing Features
Based on the column segregation in [Other Column](#column-analysis) we are designing features for different sets of groups.

<b>`feature_set1`</b> <br>
<i>Numeric features</i>
<ol>
    <li> sum </li>
    <li> mean </li>
    <li> not na rows </li>
    <li> non-zero rows </li>
    <li> max </li>
    <li> sum / leads for a client </li>
    <li> not na rows / leads for a client </li>
    <li> non-zero rows / leads for a client </li>
</ol>

<b>`feature_set2`</b><br>
<i>Datetime features</i>
<ol>
    <li> Timegap feature based on `timestamp` - gaps in minutes between first and last interaction </li>
</ol>

<b>`feature_set3`</b><br>
<i>Boolean feature</i>
<ol>
    <li> If client has `social_source_referral` </li>
</ol>

<b>`feature_set4`</b><br>
<i>Boolean features</i>
<ol>
    <li> If client has clicked on contact-us link (`contact_us_click`) and downloaded app (`app_downloads`)</li>
</ol>

<b>`feature_set5`</b><br>
<i>Categorical and Numeric features</i>
<ol>
    <li> Using the extended list of `sources` to map down to smaller feature set </li>
    <li> Finding most used `source` for a user </li>
    <li> Counting occurence of every `source` </li>
</ol>

<b>`feature_set6`</b><br>
<i>Numeric features</i>
<ol>
    <li> Count of devices (mobile, tablet) used for accessing the website </li>
</ol>

<b>`feature_set7`</b><br>
<i>Numeric features</i>
<ol>
    <li> Count of `event_category` </li>
</ol>

In [33]:
def get_not_na_count(series):
    return (~series.isna()).sum()

def get_nonzero(series):
    return ((series!=0) & (~series.isna())).sum()

def get_feature_set1(group, column, column_merge):
    # sum of column
    features = group[column].sum().reset_index()
    features.columns = [column_merge, f"{column}-sum-{column_merge}"]
    # number of not null rows
    feature2 = group[column].apply(get_not_na_count).reset_index()
    feature2.columns = [column_merge, f"{column}-count_not_na-{column_merge}"]
    features = features.merge(feature2, on=column_merge, how="outer")
    # max of column
    feature3 = group[column].max().reset_index()
    feature3.columns = [column_merge, f"{column}-max-{column_merge}"]
    features = features.merge(feature3, on=column_merge, how="outer")
    # counting non zero rows
    feature4 = group[column].apply(get_nonzero).reset_index()
    feature4.columns = [column_merge, f"{column}-count_nonzero-{column_merge}"]
    features = features.merge(feature4, on=column_merge, how="outer")
    # mean of column
    features[f"{column}-mean-{column_merge}"] = features[f"{column}-sum-{column_merge}"] / features[f"{column}-count_not_na-{column_merge}"]

    group_id_leads = group[column_id_leads].nunique().reset_index(drop=True)
    # sum/ #leads
    features[f"{column}-sum_per_lead-{column_merge}"] = features[f"{column}-sum-{column_merge}"] / group_id_leads
    # #not_na_rows/ #leads
    features[f"{column}-count_not_na_per_lead-{column_merge}"] = features[f"{column}-count_not_na-{column_merge}"] / group_id_leads
    # #not_zero_rows/ #leads
    features[f"{column}-count_nonzero_per_lead-{column_merge}"] = features[f"{column}-count_nonzero-{column_merge}"] / group_id_leads

    return features

In [34]:
def get_feature_set2(group, column, column_merge):
    # calculating time gap in minutes between first and last interaction
    features = ((group[column].max() - group[column].min()).dt.total_seconds() / 60).reset_index()
    features.columns = [column_merge, f"{column}-gap-{column_merge}"]
    return features

In [35]:
def value_in_series(value, series):
    return value in series.tolist()

def get_feature_set3(group, column, column_merge):
    # checking if "Yes" exists in a column
    features = group[column].apply(partial(value_in_series, "Yes")).reset_index()
    features.columns = [column_merge, f"{column}-value-{column_merge}"]
    return features

In [36]:
def get_feature_set4(group, column, column_merge):
    # checking if "1" exists in a column
    features = group[column].apply(partial(value_in_series, 1)).reset_index()
    features.columns = [column_merge, f"{column}-value-{column_merge}"]
    return features

In [37]:
set5_source_mapping = leads_features_info["5"]["source_mapping"]
def get_series_mode(series):
    mode = series.map(set5_source_mapping).mode()
    if len(mode):
        return mode.iloc[0]
    else:
        return np.nan

def get_series_count(value, series):
    return (series == value).sum()

def get_feature_set5(group, column, column_merge):
    # finding most used source
    features = group[column].apply(get_series_mode).reset_index()
    features.columns = [column_merge, f"{column}-mode-{column_merge}"]
    # creating a feature for each source count
    for value in set(set5_source_mapping.values()):
        feature = group[column].apply(partial(get_series_count, value)).reset_index()
        feature.columns = [column_merge, f"{column}-{value}-count-{column_merge}"]
        features = features.merge(feature, on=column_merge, how="outer")
    return features

In [38]:
def is_value(value, series):
    return (series == value).sum()

def get_feature_set6(group, column, column_merge):
    # count of devices
    values = leads_features_info["6"]["devices"]
    features = pd.DataFrame({column_merge: []})
    for value in values:
        feature = group[column].apply(partial(is_value, value)).reset_index()
        feature.columns = [column_merge, f"{column}-count_{value}-{column_merge}"]
        features = features.merge(feature, on=column_merge, how="outer")
    return features

In [39]:
def get_feature_set7(group, column, column_merge):
    # count of categories
    values = leads_features_info["7"]["categories"]
    features = pd.DataFrame({column_merge: []})
    for value in values:
        feature = group[column].apply(partial(is_value, value)).reset_index()
        feature.columns = [column_merge, f"{column}-count_{value}-{column_merge}"]
        features = features.merge(feature, on=column_merge, how="outer")
    return features

In [40]:
leads_features_info["1"]["feature_function"] = get_feature_set1
leads_features_info["2"]["feature_function"] = get_feature_set2
leads_features_info["3"]["feature_function"] = get_feature_set3
leads_features_info["4"]["feature_function"] = get_feature_set4
leads_features_info["5"]["feature_function"] = get_feature_set5
leads_features_info["6"]["feature_function"] = get_feature_set6
leads_features_info["7"]["feature_function"] = get_feature_set7

## Generating Features

### Client Features

In [41]:
feature_client = pd.DataFrame({column_client: []})
leads_client = leads_filtered.groupby(column_client)

for feature_set in leads_features_info:
    feature_set_function = leads_features_info[feature_set]["feature_function"]
    for column in leads_features_info[feature_set]["columns"]:
        print("generating features for set:", feature_set, "column:", column)
        features_set_column = feature_set_function(leads_client, column, column_client)
        feature_client = feature_client.merge(features_set_column, on=column_client, how="outer")


generating features for set: 1 column: sessions
generating features for set: 1 column: hits
generating features for set: 1 column: unique_events
generating features for set: 1 column: session_duration
generating features for set: 1 column: page_per_session
generating features for set: 1 column: enquiries
generating features for set: 1 column: pageviews
generating features for set: 1 column: time_on_page
generating features for set: 1 column: avg_time_on_page
generating features for set: 1 column: property_search
generating features for set: 1 column: lead_dropped
generating features for set: 1 column: bounce_rate
generating features for set: 1 column: total_events
generating features for set: 2 column: timestamp
generating features for set: 3 column: social_source_referral
generating features for set: 4 column: contact_us_click
generating features for set: 4 column: app_downloads
generating features for set: 5 column: source
generating features for set: 6 column: device_category
genera

In [42]:
feature_client.to_csv("./data/feature_client.csv", index=False)

In [43]:
# folder_report = "./reports/"
# if not os.path.isdir(folder_report):
#     os.makedirs(folder_report)

# path_report_features = os.path.join(folder_report, "features.html")
# profile = ProfileReport(features_client)
# profile.to_file(path_report_features)

### Lead Features

In [44]:
def get_lead_id(value):
    if isinstance(value, str):
        if value.startswith("Inventory Residential Lead - "):
            return int(value.split(" - ")[-1])
    return None
leads_filtered[f"{column_id_leads}_2"] = leads_filtered["event_action"].apply(get_lead_id)
leads_filtered[column_id_target] = leads_filtered[column_id_leads].fillna(0) + leads_filtered[f"{column_id_leads}_2"].fillna(0)

In [45]:
feature_leads = pd.DataFrame({column_id_target: []})
leads = leads_filtered.groupby(column_id_target)

for feature_set in leads_features_info:
    feature_set_function = leads_features_info[feature_set]["feature_function"]
    for column in leads_features_info[feature_set]["columns"]:
        print("generating features for set:", feature_set, "column:", column)
        features_set_column = feature_set_function(leads, column, column_id_target)
        feature_leads = feature_leads.merge(features_set_column, on=column_id_target, how="outer")

generating features for set: 1 column: sessions
generating features for set: 1 column: hits
generating features for set: 1 column: unique_events
generating features for set: 1 column: session_duration
generating features for set: 1 column: page_per_session
generating features for set: 1 column: enquiries
generating features for set: 1 column: pageviews
generating features for set: 1 column: time_on_page
generating features for set: 1 column: avg_time_on_page
generating features for set: 1 column: property_search
generating features for set: 1 column: lead_dropped
generating features for set: 1 column: bounce_rate
generating features for set: 1 column: total_events
generating features for set: 2 column: timestamp
generating features for set: 3 column: social_source_referral
generating features for set: 4 column: contact_us_click
generating features for set: 4 column: app_downloads
generating features for set: 5 column: source
generating features for set: 6 column: device_category
genera

In [46]:
feature_leads.to_csv("./data/feature_leads.csv", index=False)

### Merging `target_data` with Lead Feature on Client Features 

In [47]:
# identifying required lead IDs from `leads_data` 
lead_ids_to_consider = leads_filtered[column_id_leads].unique()

# filtering `target_data` based on required lead IDs
target_filtered = target_filtered[target_filtered[column_id_target].isin(lead_ids_to_consider)]
lead_ids_to_consider = target_filtered[column_id_target].unique()

# filtering `leads_features` based on required lead IDs
feature_leads_filtered = feature_leads[feature_leads[column_id_target].isin(lead_ids_to_consider)]

# merging `target_filtered` with Lead Features
features_combined = pd.merge(target_filtered, feature_leads_filtered, on=column_id_target, how="inner")

print("total lead IDs to consider:", len(lead_ids_to_consider))
print("shape of `target_filtered`:", target_filtered.shape)
print("shape of `feature_leads_filtered`:", feature_leads_filtered.shape)
print("shape of `features_combined`:", features_combined.shape)

total lead IDs to consider: 3334
shape of `target_filtered`: (3334, 2)
shape of `feature_leads_filtered`: (3334, 139)
shape of `features_combined`: (3334, 140)


In [48]:
# extracting lead IDs and client IDs from `leads_data` for which we have `output` information in `target_data`
client_leads = leads_filtered[[column_id_leads, column_client]].drop_duplicates()
client_leads = client_leads[client_leads[column_id_leads].isin(target_filtered[column_id_target])]
client_leads.columns = [column_id_target, column_client]

print("client-lead ID combinations in `leads_data`:", client_leads.shape[0])

client-lead ID combinations in `leads_data`: 3334


In [49]:
# merging `features_combined` with extracted lead IDs and client IDs
features = features_combined.merge(client_leads, on=column_id_target, how="outer")
print("shape of `features` after combining with client-lead ID combinations:", features.shape)

# merging `features` with Client Features
features = features.merge(feature_client, on=column_client, how="inner")
print("shape of `features` after mergining with Client Features:", features.shape)

shape of `features` after combining with client-lead ID combinations: (3334, 141)
shape of `features` after mergining with Client Features: (3334, 279)


#### To-Do
Feature Cleaning
<ol>
    <li> Dropping features with constant value throughout </li>
    <li> Dropping features with high correlation (above 90%) </li>
    <li> Dropping data points with same feature information other than `lead_id`, `client_id` and `output` values </li>
    <li> Dropping features having more than 70% of missing values </li>
</ol>

In [50]:
# dropping features with constant value throughout
feature_var_is_0 = (features.var() == 0).reset_index()
columns_with_var_0 = feature_var_is_0[feature_var_is_0[0]]["index"].tolist()
features.drop(columns=columns_with_var_0, inplace=True)

print("number of columns with zero variance:", len(columns_with_var_0))
print("shape of `features` after dropping columns with zero variance:", features.shape)

number of columns with zero variance: 68
shape of `features` after dropping columns with zero variance: (3334, 211)


C:\Users\Archisha\AppData\Local\Temp\ipykernel_10984\1935826333.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  feature_var_is_0 = (features.var() == 0).reset_index()


In [51]:
# checking correlations amongst `features`
feature_corr = features.corr().abs()
# dropping features with high correlation (above 90%)
corr_subset = feature_corr.where(np.triu(np.ones(feature_corr.shape), k=1).astype(np.bool))
columns_with_high_corr = [column for column in corr_subset.columns if any(corr_subset[column] > 0.9)]
features.drop(columns=columns_with_high_corr, inplace=True)

print("number of features with high correlation:", len(columns_with_high_corr))
print("shape of `features` after dropping columns with correlation of over 90%:", features.shape)

number of features with high correlation: 137
shape of `features` after dropping columns with correlation of over 90%: (3334, 74)


C:\Users\Archisha\AppData\Local\Temp\ipykernel_10984\2054966160.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  corr_subset = feature_corr.where(np.triu(np.ones(feature_corr.shape), k=1).astype(np.bool))


In [52]:
# dropping data points with same feature information and different `lead_id`, `client_id` and `output` values
temp = features.drop(columns=[column_id_target, column_client, column_output])
features = features.loc[temp.drop_duplicates().index]
print("shape of features after dropping duplicate data points",features.shape)

shape of features after dropping duplicate data points (2219, 74)


In [53]:
# dropping features having more than 70% of missing values
missing_values = (features.isna().mean()).reset_index()
columns_to_drop = missing_values[missing_values[0]>0.7]["index"].to_list()
features.drop(columns=columns_to_drop, inplace=True)
print("shape of final data after dropping duplicate data points",features.shape)

shape of final data after dropping duplicate data points (2219, 71)


In [54]:
# final data
features.to_csv("./data/features.csv", index=False)

# Model Selection and Training

## Data Preparation

In [55]:
data = features.copy()

# finding categorical columns in the data 
categorical_columns = [column for column in data.columns if not(is_numeric_dtype(data[column]))]
print("categorical columns in data:", categorical_columns)

categorical columns in data: ['source-mode-client_id']


In [56]:
# label encoding categorical columns
label_encoder = {}

for column in categorical_columns:
    label_encoder[column] = preprocessing.LabelEncoder()
    data[column].fillna("0", inplace=True)
    data[column] = label_encoder[column].fit_transform(data[column])


In [57]:
# defining features and target with X and y
X = data.drop(columns=[column_output]).copy()
y = data[column_output]

# creating train-val-test split
# splitting data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.8, random_state=226)

# spliting remaining data into equal sections of validation and test split (15% each of overall data). 
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=226)

print("shape of train data:", X_train.shape, y_train.shape)
print("shape of validation data:", X_valid.shape, y_valid.shape)
print("shape of test data:", X_test.shape, y_test.shape)

shape of train data: (1775, 70) (1775,)
shape of validation data: (222, 70) (222,)
shape of test data: (222, 70) (222,)


In [58]:
# dropping `client_id` and `lead_id` from train data, valid data and test data
X_train.drop(columns=[column_id_target, column_client], inplace=True)

X_valid_index = X_valid[[column_id_target, column_client]]
X_test_index = X_test[[column_id_target, column_client]]

X_valid.drop(columns=[column_id_target, column_client], inplace=True)
X_test.drop(columns=[column_id_target, column_client], inplace=True)

In [59]:
# filling missing values with default value -1
X_train.fillna(-1, inplace=True)
X_valid.fillna(-1, inplace=True)
X_test.fillna(-1, inplace=True)

# using SMOTE to compensate for data scarcisty
oversampling = SMOTE(random_state=226)
X_train, y_train = oversampling.fit_resample(X_train, y_train)

print("shape of train data:", X_train.shape, y_train.shape)

shape of train data: (2934, 68) (2934,)


In [60]:
# model scoring function - calculating f1 and roc-auc scores
def scoring(model):
    results = {}

    y_predict = model.predict(X_train)
    results["train-f1"] = f1_score(y_predict, y_train)
    results["train-roc_auc"] = roc_auc_score(y_predict, y_train)

    y_predict = model.predict(X_valid)
    results["valid-f1"] = f1_score(y_predict, y_valid)
    results["valid-roc_auc"] = roc_auc_score(y_predict, y_valid)

    y_predict = model.predict(X_test)
    results["test-f1"] = f1_score(y_predict, y_test)
    results["test-roc_auc"] = roc_auc_score(y_predict, y_test)

    return results

## Baseline Model - Logistic Regression

In [61]:
# defining a list to store all model results
model_results = []

In [62]:
# training logistic regression with no hyperparameter tuning as the baseline model
baseline_dt = LogisticRegression(random_state=0)
baseline_dt.fit(X_train, y_train)

baseline_dt_score = scoring(baseline_dt)
print("F1 and ROC-AUC score for train, validation and test data:", baseline_dt_score)

result_df = pd.DataFrame(baseline_dt_score.items()).set_index(0).T.reset_index()
result_df["model"] = "Logistic Regression"
result_df["params"] = "baseline"
model_results.append(result_df)

F1 and ROC-AUC score for train, validation and test data: {'train-f1': 0.6305170239596469, 'train-roc_auc': 0.6032632686613105, 'valid-f1': 0.36486486486486486, 'valid-roc_auc': 0.570755865876431, 'test-f1': 0.3096774193548387, 'test-roc_auc': 0.5358974358974359}


c:\Users\Archisha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Decision Tree

In [63]:
# training decision tree with hyperparameter tuning and storing the results for model benchmarking
print("calculating F1 and ROC-AUC score for train, validation and test data along with different hyperparameter values")

for depth in range(4, 15):
    tree_dt = DecisionTreeClassifier(max_depth=depth, random_state=0)
    tree_dt.fit(X_train, y_train)
    
    result = scoring(tree_dt)
    params = f"depth:{depth}"

    result_df = pd.DataFrame(result.items()).set_index(0).T.reset_index()
    result_df["model"] = "Decision Tree"
    result_df["params"] = params
    model_results.append(result_df)
    

calculating F1 and ROC-AUC score for train, validation and test data along with different hyperparameter values


## K-Nearest Neighbours

In [64]:
# training KNN with hyperparameter tuning and storing the results for model benchmarking
print("calculating F1 and ROC-AUC score for train, validation and test data along with different hyperparameter values")

neighbors = [10, 20, 50, 80, 100, 150]
leaf_sizes = [10, 20, 50]
for neighbor in neighbors:
    for leaf_size in leaf_sizes:
        nei_knn = KNeighborsClassifier(n_neighbors = neighbor, weights='uniform',leaf_size=leaf_size)
        nei_knn.fit(X_train, y_train)

        result = scoring(nei_knn)
        params = f"neighbor:{neighbor}, leaf_size:{leaf_size}"

        result_df = pd.DataFrame(result.items()).set_index(0).T.reset_index()
        result_df["model"] = "KNN"
        result_df["params"] = params
        model_results.append(result_df)


calculating F1 and ROC-AUC score for train, validation and test data along with different hyperparameter values


## Gradient Boost

In [65]:
print("calculating F1 and ROC-AUC score for train, validation and test data along with different hyperparameter values")

learning_rates = [0.01, 0.05, 0.1]
n_estimators = [10, 25, 50, 100, 150]
min_samples_splits = [10, 25, 50]
max_depths = [4, 7, 10, 15]

for learning_rate in learning_rates:
    for n_estimator in n_estimators:
        for min_samples_split in min_samples_splits:
            for max_depth in max_depths:
                model_gb = GradientBoostingClassifier(
                    random_state = 0, 
                    n_estimators = n_estimator, 
                    learning_rate = learning_rate,
                    min_samples_split = min_samples_split,
                    max_depth = max_depth
                )

                model_gb.fit(X_train, y_train)

                result = scoring(model_gb)
                params = f"learning_rate:{learning_rate}, n-estimators:{n_estimator}, min_samples_split:{min_samples_split}, max_depth:{max_depth}"

                result_df = pd.DataFrame(result.items()).set_index(0).T.reset_index()
                result_df["model"] = "GBM"
                result_df["params"] = params
                model_results.append(result_df)


calculating F1 and ROC-AUC score for train, validation and test data along with different hyperparameter values


## Random Forest

In [66]:
print("calculating F1 and ROC-AUC score for train, validation and test data along with different hyperparameter values")

ne_list = [10, 20, 50, 80, 100, 150, 200]

for est in ne_list:
    for depth in range(4, 10):

        tree_rf = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=0)
        tree_rf.fit(X_train, y_train)
        
        result = scoring(tree_rf)
        params = f"n_estimators:{est}, max_depth:{depth}"

        result_df = pd.DataFrame(result.items()).set_index(0).T.reset_index()
        result_df["model"] = "Random Forest"
        result_df["params"] = params
        model_results.append(result_df)    


calculating F1 and ROC-AUC score for train, validation and test data along with different hyperparameter values


# Benchmarking Model Results

In [67]:
# concating all the model results
model_eval_df = pd.concat(model_results, ignore_index=True)
df_columns = model_eval_df.columns[1:].tolist()
model_eval_df = model_eval_df[df_columns[-2:] + df_columns[:-2]]

In [79]:
# identifying the best model 
model_eval_df[(model_eval_df["valid-roc_auc"]>0.62) & (model_eval_df["valid-f1"]>0.42)]

,model,params,train-f1,train-roc_auc,valid-f1,valid-roc_auc,test-f1,test-roc_auc
30,GBM,"learning_rate:0.01, n-estimators:10, min_sampl...",0.732240,0.732792,0.431373,0.621144,0.330579,0.555734
34,GBM,"learning_rate:0.01, n-estimators:10, min_sampl...",0.732240,0.732792,0.431373,0.621144,0.330579,0.555734
38,GBM,"learning_rate:0.01, n-estimators:10, min_sampl...",0.732240,0.732792,0.431373,0.621144,0.330579,0.555734
222,Random Forest,"n_estimators:50, max_depth:4",0.770349,0.789043,0.426667,0.660394,0.380952,0.611413
234,Random Forest,"n_estimators:100, max_depth:4",0.774542,0.789447,0.430380,0.652060,0.417582,0.623032
235,Random Forest,"n_estimators:100, max_depth:5",0.809456,0.821692,0.422535,0.671217,0.400000,0.620851
240,Random Forest,"n_estimators:150, max_depth:4",0.773754,0.787746,0.435897,0.657839,0.422222,0.626810
246,Random Forest,"n_estimators:200, max_depth:4",0.769010,0.783311,0.441558,0.663902,0.413043,0.619378


## Observations
To get the best two models we are looking at `valid-roc_auc` greater than 62% and from above result we can conclude 
* Random Forest model with 100 n-estimators and 5 as the maximum depth
* Gradient Boost model with learning_rate=0.01, n-estimators=10, min_samples_split=25 and max_depth=4

are the best and avoid cases of overfitting

# Testing and Scoring

In [69]:
# finding leads conversion rate without using machine learning
default_success_percentage = 100 * y_valid.mean()
print("success percentage without machine learning:", default_success_percentage)

success percentage without machine learning: 17.56756756756757


In [70]:
def get_prediction_probability(model, x):
    return model.predict_proba(x)[:, 1]

In [86]:
# finding leads conversion rate using machine learning
final_models = {}

final_models["Random Forest"] = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
final_models["Random Forest"].fit(X_train, y_train)

final_models["Gradient Boosting Model"] = GradientBoostingClassifier(learning_rate=0.01, n_estimators=10, min_samples_split=25, max_depth=4, random_state=0)
final_models["Gradient Boosting Model"].fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.01, max_depth=4,
                           min_samples_split=25, n_estimators=10,
                           random_state=0)

In [87]:
for model_name, model in final_models.items():
    # scoring leads and identifying top picks
    scoring_df = pd.DataFrame({
        column_id_target: X_valid_index[column_id_target],
        "lead_success_score": get_prediction_probability(model, X_valid),
        "ground_truth": y_valid
    }).sort_values(by="lead_success_score", ascending=False)

    top_picks = [50, 100]
    for top_pick in top_picks:
        success_percentage = 100 * scoring_df.iloc[:top_pick]["ground_truth"].mean()
        print(f"success percentage for top {top_pick} picks using {model_name}:", success_percentage)
        print(f"we can increase leads conversion rate for top {top_pick} picks by:", success_percentage - default_success_percentage)
        print("===================================================")

scoring_df.head(5)

success percentage for top 50 picks using Random Forest: 36.0
we can increase leads conversion rate for top 50 picks by: 18.43243243243243
success percentage for top 100 picks using Random Forest: 27.0
we can increase leads conversion rate for top 100 picks by: 9.432432432432432
success percentage for top 50 picks using Gradient Boosting Model: 40.0
we can increase leads conversion rate for top 50 picks by: 22.43243243243243
success percentage for top 100 picks using Gradient Boosting Model: 24.0
we can increase leads conversion rate for top 100 picks by: 6.432432432432432


,lead_id,lead_success_score,ground_truth
2386,299054,0.539002,1
2250,298769,0.539002,0
7,287846,0.513764,0
2557,290481,0.513764,1
922,296518,0.513764,1


## Observation
From the above results on validation data we conclude that Random Forest Model performs better than Gradient Boosting Model <br>

<b> Model selected - Random Forest Model </b>

## Scoring on test data (using selected model)

In [94]:
test_success_percentage = 100 * y_test.mean()

model_name = "Random Forest"
final_model = final_models[model_name]
# scoring leads using probablities and identifying top picks
scoring_df = pd.DataFrame({
    column_id_target: X_test_index[column_id_target],
    "lead_success_score": get_prediction_probability(final_model, X_test),
    "ground_truth": y_test
}).sort_values(by="lead_success_score", ascending=False)

top_pick = 50
success_percentage = 100 * scoring_df.iloc[:top_pick]["ground_truth"].mean()
print("success percentage on test set without machine learning:", test_success_percentage)
print(f"success percentage for top {top_pick} test picks using {model_name}:", success_percentage)
print(f"we can increase leads conversion rate for top {top_pick} test picks by:", success_percentage - test_success_percentage)


success percentage on test set without machine learning: 17.117117117117118
success percentage for top 50 test picks using Random Forest: 36.0
we can increase leads conversion rate for top 50 test picks by: 18.882882882882882


## Identifying Important Features

In [92]:
feature_importace = pd.DataFrame({"column": X_train.columns, "importance": final_model.feature_importances_})
feature_importace.sort_values(by="importance", ascending=False, inplace=True)
feature_importace.iloc[:10]

,column,importance
17,sessions-sum_per_lead-client_id,0.078356
43,bounce_rate-max-client_id,0.074881
42,bounce_rate-sum-client_id,0.062534
9,lead_dropped-mean-lead_id,0.056392
44,bounce_rate-mean-client_id,0.048686
22,session_duration-mean-client_id,0.042267
21,session_duration-max-client_id,0.040844
50,source-mode-client_id,0.034769
46,timestamp-gap-client_id,0.034032
38,property_search-mean-client_id,0.029067


# Conclusion

* We have successfully built a scoring mechanism that will help the sales executives prioritize the leads using Random Forest
* Originally the data had a 17% success rate of converting leads to tenants
* Using this model we can achieve a 19% increase, i.e a total of 36% success rate in lead to tenant conversion while selecting top 50 leads
* Insights for the client: Recording more information on the following will help online behavior analysis and improve scoring model
    * Duration of sessions directly affects the conversion rate 
    * Number of sessions directly affects the conversion rate
    * Bounce rate inversely affects the conversion rate
    * Some categories of `Source` directly affects the conversion rate
    * Lead ID assignment methodology - understand and refine the logic for assigning lead IDs eg: a lead ID is assigned to only some rows of the same client even though the others contain same information